In [1]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import KFold, cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from six.moves import cPickle as pickle
import numpy as np
import time
import datetime
import pandas as pd

features = pd.read_csv('train/train_small.csv', header = 0)
X = features.drop(['order_id','cancel', 'dow'], axis=1)
y = features['cancel']
print('{0} train samples loaded'.format(len(X)))

#print(X.head())

51761 train samples loaded


# Preprocessing

In [2]:
#def fill_na_zero(df, col_name):
#    df[col_name] = df[col_name].fillna(df[col_name].mean())
#    df = df.fillna(0)
#    #df = df.fillna(df.mean())
#    return df

def fill_na_mean(df, col_names):
    for c in col_names:
        df[c] = df[c].fillna(df[c].mean())
    return df

def le_fit_transform(df, col_name):
    le = LabelEncoder()
    le.fit(df[col_name])
    df[col_name] = le.transform(df[col_name])
    return df, le

def le_transform(df, col_name, le):
    df[col_name] = le.transform(df[col_name])
    return df

def ohc_fit_transform(df, col_name):
    ohe = OneHotEncoder(sparse=False)
    ohe.fit(df[col_name].values.reshape(-1, 1))
    encoded_col = ohe.transform(df[col_name].values.reshape(-1, 1))
    #encoded_col = ohe.fit_transform(df[col_name].values.reshape(-1, 1))
    tmp = pd.DataFrame(encoded_col, columns=[col_name + str(i) for i in range(encoded_col.shape[1])], index = df.index)
    df = pd.concat([df, tmp], axis = 1)
    df = df.drop([col_name], axis = 1)
    return df, ohe

def ohc_transform(df, col_name, ohe):
    encoded_col = ohe.transform(df[col_name].values.reshape(-1, 1))
    tmp = pd.DataFrame(encoded_col, columns=[col_name + str(i) for i in range(encoded_col.shape[1])], index = df.index)
    df = pd.concat([df, tmp], axis = 1)
    df = df.drop([col_name], axis = 1)
    return df

def exponentiation(df, cols):
    for c in cols:
        df[c + '_sq'] = df[c]**2
        df[c + '_sqrt'] = np.sqrt(df[c])
    return df
        

 
X = fill_na_mean(X, ['shifts_num', 'dow_paid_share'])
X = X.fillna(0)


#X, le_type = le_fit_transform(X, 'type')
#X, ohc_type = ohc_fit_transform(X, 'type')

#X, le_start_hour = le_fit_transform(X, 'start_hour')
#X, ohc_le_start_hour = ohc_fit_transform(X, 'start_hour')

#X, le_creation_mean = le_fit_transform(X, 'creation_mean')
#X, ohc_creation_mean = ohc_fit_transform(X, 'creation_mean')

#X, le_payment_type = le_fit_transform(X, 'payment_type')
#X, ohc_payment_type = ohc_fit_transform(X, 'payment_type')

# X = exponentiation(X, ['paid_all','canceled_all_nt','paid_16d', 'paid_30d', 'paid_60d', 'canceled_16d_nt','canceled_30d_nt', 'canceled_60d_nt', 'dow_paid', 'dow_canceled'])

In [3]:
folds = 5
kf = KFold(len(X), folds, shuffle=True, random_state=42)
trees = [50, 75, 100]
depths = [2, 3, 4, 5, 6]
rates = [0.1, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]


print()
print('Gradient boosting fitting:')

for t in trees:
    for d in depths:
        for r in rates:
            start_time = datetime.datetime.now()
            gb_clf = GradientBoostingClassifier(learning_rate=r, n_estimators=t, verbose=False, random_state=241, max_depth = d)
            scores = cross_val_score(gb_clf, X=X, y=y, scoring='roc_auc', cv=kf)
            time_to_fit = datetime.datetime.now() - start_time
            print('Trees {0}, Depth {1}, Learning Rate {2}, Time to fit: {3}, ROC-AUC: {4}'.format(t, d, r, time_to_fit, scores.mean()))


Gradient boosting fitting:
Trees 50, Depth 2, Learning Rate 0.1, Time to fit: 0:00:11.772328, ROC-AUC: 0.7924604546148322
Trees 50, Depth 2, Learning Rate 0.2, Time to fit: 0:00:11.866682, ROC-AUC: 0.8018385583767442
Trees 50, Depth 2, Learning Rate 0.25, Time to fit: 0:00:13.183852, ROC-AUC: 0.8046883820845375
Trees 50, Depth 2, Learning Rate 0.3, Time to fit: 0:00:11.664908, ROC-AUC: 0.8054996252211089
Trees 50, Depth 2, Learning Rate 0.35, Time to fit: 0:00:09.307474, ROC-AUC: 0.8066745446357408
Trees 50, Depth 2, Learning Rate 0.4, Time to fit: 0:00:09.015783, ROC-AUC: 0.8072625553883632
Trees 50, Depth 2, Learning Rate 0.45, Time to fit: 0:00:09.395578, ROC-AUC: 0.8073094025412699
Trees 50, Depth 2, Learning Rate 0.5, Time to fit: 0:00:09.266642, ROC-AUC: 0.8077318166929175
Trees 50, Depth 3, Learning Rate 0.1, Time to fit: 0:00:17.612166, ROC-AUC: 0.8048400121279219
Trees 50, Depth 3, Learning Rate 0.2, Time to fit: 0:00:16.019580, ROC-AUC: 0.8112854845744086
Trees 50, Depth 3, 

Trees 100, Depth 2, Learning Rate 0.45, Time to fit: 0:00:17.062252, ROC-AUC: 0.8116962263808501
Trees 100, Depth 2, Learning Rate 0.5, Time to fit: 0:00:16.905144, ROC-AUC: 0.8114893088793597
Trees 100, Depth 3, Learning Rate 0.1, Time to fit: 0:00:30.784309, ROC-AUC: 0.8119034248270675
Trees 100, Depth 3, Learning Rate 0.2, Time to fit: 0:00:29.093160, ROC-AUC: 0.8153423126939201
Trees 100, Depth 3, Learning Rate 0.25, Time to fit: 0:00:28.672846, ROC-AUC: 0.8159449855080364
Trees 100, Depth 3, Learning Rate 0.3, Time to fit: 0:00:28.535720, ROC-AUC: 0.8159411346098148
Trees 100, Depth 3, Learning Rate 0.35, Time to fit: 0:00:28.154633, ROC-AUC: 0.8159509590854028
Trees 100, Depth 3, Learning Rate 0.4, Time to fit: 0:00:28.196763, ROC-AUC: 0.8154404850860288
Trees 100, Depth 3, Learning Rate 0.45, Time to fit: 0:00:27.846672, ROC-AUC: 0.8145576175318681
Trees 100, Depth 3, Learning Rate 0.5, Time to fit: 0:00:27.631544, ROC-AUC: 0.8143640994430774
Trees 100, Depth 4, Learning Rate 0.

KeyboardInterrupt: 

# Model Training

In [ ]:
final_gb_clf = GradientBoostingClassifier(learning_rate=0.2, n_estimators=75, max_depth=5, verbose=False, random_state=241)
final_gb_clf.fit(X, y)


coeff = final_gb_clf.feature_importances_
names = X.columns.values
df = pd.DataFrame({'features':names, 'importance':coeff})
print(df.sort('importance'))

In [207]:
le_creation_mean.classes_

array(['admin', 'auto', 'crm', 'ios', 'reactivation', 'web_new_flow'], dtype=object)

### Dump model to pickle

In [7]:
#file_name = 'cancellations_model.pickle'
#with open(file_name, 'wb') as f:
#    pickle.dump(final_gb_clf, f, pickle.HIGHEST_PROTOCOL)

In [4]:
pickle_file = 'cancellations_model.pickle'
try:
    f = open(pickle_file, 'wb')
    save = {
        'model': final_gb_clf,
        #'ohc_dow': ohc_dow,
        #'le_type': le_type,
        #'ohc_type': ohc_type,
        #'le_creation_mean': le_creation_mean,
        #'ohc_creation_mean': ohc_creation_mean,
        #'le_payment_type': le_payment_type,
        #'ohc_payment_type': ohc_payment_type,
    }
    pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
    f.close()
except Exception as e:
    print('Unable to save data to', pickle_file, ':', e)
    raise

# Тест на отложенной выборке

In [217]:
features = pd.read_csv('test/test-20-29-jun.csv', header = 0)

X_test = features.drop(['order_id','cancel', 'dow'], axis=1)


# X_test = fill_na(X_test, 'shifts_num')

X_test = fill_na_mean(X_test, ['shifts_num', 'dow_paid_share'])

X_test = X_test.fillna(0)




#X_test = ohc_transform(X_test, 'dow', ohc_dow)

#X_test = le_transform(X_test, 'type', le_type)
#X_test = ohc_transform(X_test, 'type', ohc_type)

#X_test = le_transform(X_test, 'creation_mean', le_creation_mean)
#X_test = ohc_transform(X_test, 'creation_mean', ohc_creation_mean)

#X_test = le_transform(X_test, 'payment_type', le_payment_type)
#X_test = ohc_transform(X_test, 'payment_type', ohc_payment_type)

# X_test = exponentiation(
#    X_test, ['paid_all','canceled_all','paid_16d','canceled_16d', 'paid_30d','canceled_30d', 'paid_60d','canceled_60d'])


y_true = features['cancel']
y_pred = final_gb_clf.predict_proba(X_test)[:,1]


print(roc_auc_score(y_true, y_pred))

0.82317516594


### Bins

In [58]:
bins = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
group_names = ['0-10', '10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80', '80-90', '90-100']

categories = pd.cut(y_pred, bins, labels=group_names)

df = pd.DataFrame(columns=['y_true', 'y_pred', 'bin'])
df['y_true'] = y_true
df['y_pred'] = y_pred
df['bin'] = categories

df.to_csv('bins_nt_model-2017-06-29.csv')

# Prediction

In [218]:
features = pd.read_csv('predict/2017-07-01.csv', header = 0, index_col='order_id')
#X_test = read_csv('test/test-2017.csv', index_col=[0,1,2,3])




# Preprocessing
X_test = features

X_test = features.drop(['dow'], axis=1)
X_test = fill_na_mean(X_test, ['shifts_num', 'dow_paid_share'])
X_test = X_test.fillna(0)



#y_true = features['cancel']
y_pred = final_gb_clf.predict_proba(X_test)[:,1]


#print(roc_auc_score(y_true, y_pred))
print(y_pred)


df = pd.DataFrame(index=X_test.index, columns=['will_cancel'])
df['will_cancel'] = final_gb_clf.predict_proba(X_test)[:,1]
df.to_csv('predict/predict-2017-07-01.csv')
print('DONE')

[ 0.47702018  0.50489465  0.47570555 ...,  0.5580376   0.10706268
  0.17947012]
DONE
